In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

##### [1. Forward Imaging Model](fwdimaging.ipynb)

## 2. The Recovery Process

This process recovers the Hi-Res complex object using the Lo-Res image set. Each image of it is generated by the lighting that emerges from a different LED of a LED illuminator during the forward imaging process ([fwdimaging.ipynb](fwdimaging.ipynb)).

In [ ]:
import fwdimaging as fwd
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import os
import math

In [ ]:
%matplotlib inline

In [ ]:
def get_sequence(leds_prc, **kwargs):
    """Returns a list with the indexes of the images for the
    reconstruction sequence.
    
    The way is spiral-shaped. For squared arrays with odd number of
    rows, the spiral's head is the center; instead, for an even
    number, the head is in the (center + 1, center - 1) coordinate.
    """
    clockwise = kwargs.pop('clockwise', True)
    n2max = kwargs.pop('n2max', leds_prc ** 2 - 1)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_idx_seq_img.__name__, list(kwargs.keys())[-1]))
    l_idx_img_seq = list()
    if leds_prc % 2 != 0:
        img = int(((leds_prc ** 2) - 1) / 2)  # Start in the center.
    else:
        img = int(((leds_prc ** 2) - leds_prc) / 2)
    l_idx_img_seq.append(img)
    l_addends = [(leds_prc, -1), (-leds_prc, 1)]
    if not clockwise:
        l_addends = [(leds_prc, 1), (-leds_prc, -1)]
    # Index to choose between (leds_prc, -1) ('tuple') and
    # (-leds_prc, 1).
    i = 0
    c = 1
    while True:
        i %= 2
        v_addend, h_addend = l_addends[i]
        for j in range(c):
            img += v_addend
            if img > n2max or img < 0 or img in l_idx_img_seq:
                return l_idx_img_seq
            else:
                l_idx_img_seq.append(img)
        for j in range(c):
            img += h_addend
            if img > n2max or img < 0 or img in l_idx_img_seq:
                return l_idx_img_seq
            else:
                l_idx_img_seq.append(img)
        i += 1
        c += 1

In [ ]:
def get_photo(**kwargs):
    """Returns the Hi-Res object recovered by applying of Fourier
    Ptychography algorithm."""
    inpath = kwargs.pop('inpath',
                        os.path.join(os.path.expanduser('~'),
                                     's3-out',
                                     'microscope',
                                     'lores-set-0001'))
    outdirname = kwargs.pop('outdirname', os.path.join(inpath,
                                                       'hq-fp-0001'))
    outpath = kwargs.pop('outpath', fwd.find_out_dir(dirname=outdirname,
                                    parentdir='microscope'))
    inprefix = kwargs.pop('prefix', 'lores-img_')
    ext = kwargs.pop('ext', '.tif')  # Extension of the input files.
    firstsuffix = kwargs.pop('firstsuffix', '001')
    illuminator = kwargs.pop('illuminator', 'GRID')
    # For a LED illuminator type 'GRID', leds ('int') is the number of
    # LEDs per row or per column of a squared LED grid, e.g. 15;
    # instead, for a one type 'RING', leds is a list that contains the
    # number of LEDs by ring from the center to the edge of the LED
    # illuminator, e.g., [12, 18, 36].
    leds = kwargs.pop('leds', 15)
    # leds2use ('int') is the ORDER OF A SQUARED SUBSET of Lo-Res image
    # set. This subset is aligned to the center of the main set.
    # leds2use must be less than or equal to n ('int').
    leds2use = kwargs.pop('leds2use', -1)
    d = kwargs.pop('d', 4)  # Distance in mm between LEDs.
    # Distance in mm between the LED grid and the sample.
    h = kwargs.pop('h', 90)
    # Indexes in Matlab begin in 1. Does the first image have 1 at the
    # end of its filename?
    namefrom1 = kwargs.pop('namefrom1', True)
    ##
    # Determination of default valor for subsel ('list').
    l_subsel_idxs = list()
    if isinstance(leds, int):  # Default subset for illuminator type
                               # 'GRID'.
        if leds2use == -1 or leds2use > leds:
            leds2use = leds
        l_subsel_idxs = fwd.get_centered_img_subset(leds, leds2use,
                                                    matlab=namefrom1)
    elif isinstance(leds, list):  # Default subset for illuminator
                                  # type 'RING'.
        n_leds = math.ceil(math.sqrt(sum(leds)))
        if leds2use == -1 or leds2use > n_leds:
            leds2use = n_leds
        l_subsel_idxs = fwd.get_centered_img_subset(n_leds, leds2use,
                                                    matlab=namefrom1)
    subsel = kwargs.pop('subsel', l_subsel_idxs)
    ##
    # Parameters of the coherent imaging system.
    wavelen = kwargs.pop('wavelen', 0.63e-6)
    # Sampling pixel size of the CCD.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    # DANGER! IT COULD PIXELATE the output image.
    zoom = kwargs.pop('zoom', 4)
    loops = kwargs.pop('loops', 5)
    # Show the output Hi-Res image in the notebook.
    hqname = kwargs.pop('hqname', 'hq-img')  # Name of the file.
    # Does it the output image in the notebook of the calling?
    show = kwargs.pop('show', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_photo.__name__, list(kwargs.keys())[-1]))
    l_xy_leds = list()
    if illuminator.lower() == 'grid' and isinstance(leds, int):
        l_xy_leds = fwd.gen_xy_led_grid(leds2use, d)
    elif illuminator.lower() == 'ring' and isinstance(leds, list):
        l_xy_leds = fwd.gen_xy_led_ring(leds, d)
    wvs = fwd.generate_wave_vectors(l_xy_leds, h)
    lores_imgs_seq, zeros = fwd.get_set_from_folder(subsel,
                                                    path=inpath,
                                                    prefix=inprefix,
                                                    extension=ext,
                                                    lensuffix=len(firstsuffix))
    # Search for the height and the width in pixels of a Lo-Res
    # picture.
    hpx = 0
    wpx = 0
    for i in range(leds2use ** 2):
        if len(lores_imgs_seq[i]) != 0:
            hpx, wpx = lores_imgs_seq[i].shape
            break
    # np.zeros((0, 0)) arrays in lores_imgs_seq ('numpy.ndarray') are
    # replaced by other array from the same kind and wpx-by-hpx shape.
    for i in zeros:
        lores_imgs_seq[i] = np.zeros((hpx, wpx))
    hpx *= zoom  # Height of the output Hi-Res image.
    wpx *= zoom  # ... the width.
    p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx,
                                              zoom, na, wpx, hpx)
    # For a shifted subset, the -1 values will also be considered 
    seq = get_sequence(leds2use)
    obj_recover = np.ones((hpx, wpx), dtype=np.int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in range(leds2use ** 2):
            k = seq[j]
            kxc = fwd.round_half_up((wpx + 1) / 2.0 + kx[k] / dkx)
            kyc = fwd.round_half_up((hpx + 1) / 2.0 + ky[k] / dky)
            kyl = fwd.round_half_up(kyc - (p - 1) / 2.0)
            kyh = fwd.round_half_up(kyc + (p - 1) / 2.0)
            kxl = fwd.round_half_up(kxc - (q - 1) / 2.0)
            kxh = fwd.round_half_up(kxc + (q - 1) / 2.0)
            lores_ft = ((p / hpx) ** 2) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh]
            lores_ft *= cft
            img_lores = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            img_lores = ((hpx / p) ** 2) * lores_imgs_seq[k] * np.exp(1j * np.angle(img_lores))
            lores_ft = np.fft.fftshift(np.fft.fft2(img_lores)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = (1 - cft) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft
    filename = hqname + ext
    img_path = os.path.join(outpath, filename)
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    scipy.misc.toimage(obj_recover, cmin=0, cmax=255).save(img_path)
    print('Saved high-quality image:', img_path)
    if show:
        plt_img = plt.imshow(obj_recover, cmap='Greys_r')
    return obj_recover